# Lending Club Case Study

## **1. Introduction**



### **1.1. Objective**
Analyse the given data set of a leading lending company, identify the risks and issues with the applicant's borrowing pattern. Prepare a detailed case study document highlighting the risks involved and provide my observation and recommendations.

### **1.2. Problem Statement**
- Make a decision on a loan application when it's received based on the risk factors
- Should not reject an application if the applicant has the potential to repay the loan
- Should not approve an application if the applicant could possibly default

## **2. Data Understanding**



### **2.1. Import necessary libraries**


In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Pandas by default doesn't display all the columns in the dataframe
# As we're going to work on a large dataset, the following setting will help read data from all the columns
pd.set_option('display.max_columns', None)

# For some of the columns we may have to see the data from all rows
# Eg: Categorical columns.
pd.set_option('display.max_rows', None)

# Set themes, styles and color palette for seaborn charts
sns.set_theme(style='darkgrid', context='paper')
sns.set_palette(palette='pastel')

plt.figure(figsize=(12, 4))

plt.rcParams["axes.titlesize"] = 14 
plt.rcParams["axes.labelsize"] = 12 
plt.rcParams["xtick.labelsize"] = 10 
plt.rcParams["ytick.labelsize"] = 10 

<Figure size 1200x400 with 0 Axes>

### **2.2. Data Overview**

In [31]:
# Load the data from loan.csv file. 
# Using the read_csv function from the pandas library, we can load the data from the csv to a pandas dataframe.
df = pd.read_csv('Data/loan.csv', low_memory=False)

# Shape property of the pandas dataframe returns the dimention of the dataframe
# The return value will contain the number of rows and columns in the dataframe
df.shape

# Quick observation
#-------------------
# 1. There are 39717 rows and
# 2. There are 111 columns
#-------------------

(39717, 111)

In [32]:
df.dtypes

# Looking into the dtypes property of the dataframe, help us understand the data type of each column
#
# Quick observation
#-------------------
# 1. Most of the columns are of type object.
# 2. `issue_d` looks like a date column. Also other columns with `_d` suffix.
# 3. `grade`, `sub_grade`, `term`, `loan_status`, `verification_status` etc. looks like category columns
# 4. Columns like `id`, `member_id`, `url`, `desc` may not be useful for analysing the risk. Hence can be removed.
#-------------------

id                                  int64
member_id                           int64
loan_amnt                           int64
funded_amnt                         int64
funded_amnt_inv                   float64
term                               object
int_rate                           object
installment                       float64
grade                              object
sub_grade                          object
emp_title                          object
emp_length                         object
home_ownership                     object
annual_inc                        float64
verification_status                object
issue_d                            object
loan_status                        object
pymnt_plan                         object
url                                object
desc                               object
purpose                            object
title                              object
zip_code                           object
addr_state                        

In [33]:
# Pandas dataframe object offers `nunique` function which give the number of unique elements in each column
df.nunique().sort_values(ascending=False)

# Quick observation
#-------------------
# 1. Some columns have unique values equal to the number of columns (39717)
# 2. Some columns have 0 unique values! Must be all empty
# 3. Both such columns are not going to help us with data analysis
# 4. Columns `loan_amnt`, `funded_amnt` and `funded_amnt_inv` says the borrower applied for an amount, but the institution decided to fund either more or less.
#    For our analysis we only need the amount the borrower applied for. Hence the investor related columns can be removed before we proceed furthen.
# 5. While `addr_state` has 50 unique values, the `zip_code` only has 823 unique values (Proportionally less). There's something going on with `zip_code`.
#-------------------

id                                39717
url                               39717
member_id                         39717
total_pymnt                       37850
total_pymnt_inv                   37518
total_rec_int                     35148
last_pymnt_amnt                   34930
emp_title                         28820
desc                              26526
revol_bal                         21711
title                             19615
installment                       15383
funded_amnt_inv                    8205
total_rec_prncp                    7976
annual_inc                         5318
recoveries                         4040
dti                                2868
collection_recovery_fee            2616
total_rec_late_fee                 1356
out_prncp_inv                      1138
out_prncp                          1137
revol_util                         1089
funded_amnt                        1041
loan_amnt                           885
zip_code                            823


In [34]:
# Get the columns in the decending order of most number of null values 
print(df.isnull().sum().sort_values(ascending=False))

# Quick observation
#-------------------
# 1. Some columns have null or empty values equal to the number of columns (39717)
# 2. Such columns are not going to help us with data analysis
#-------------------

verification_status_joint         39717
annual_inc_joint                  39717
mo_sin_old_rev_tl_op              39717
mo_sin_old_il_acct                39717
bc_util                           39717
bc_open_to_buy                    39717
avg_cur_bal                       39717
acc_open_past_24mths              39717
inq_last_12m                      39717
total_cu_tl                       39717
inq_fi                            39717
total_rev_hi_lim                  39717
all_util                          39717
max_bal_bc                        39717
open_rv_24m                       39717
open_rv_12m                       39717
il_util                           39717
total_bal_il                      39717
mths_since_rcnt_il                39717
open_il_24m                       39717
open_il_12m                       39717
open_il_6m                        39717
open_acc_6m                       39717
tot_cur_bal                       39717
tot_coll_amt                      39717


In [35]:
# Get those column which has null values accross the series
print(df.columns[df.isnull().all()])

# Let us find out how many columns has null values in all the rows.
print(f"Number of columns with null values in all the rows: {df.columns[df.isnull().all()].size}")

# Quick observation
#-------------------
# 1. We have 54 columns which has null values in all the rows
# 2. We can safely remove them
#-------------------

Index(['mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'tot_coll_amt', 'tot_cur_bal',
       'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m',
       'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m',
       'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi',
       'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal',
       'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
       'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
       'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m',
       'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m',
       'pct_tl_nvr_dl

In [ ]:
# Get those column which has null values is some rows but not all
print(df.columns[~df.isnull().all() & df.isnull().any()])

someButNotAllNulls = df.columns[~df.isnull().all() & df.isnull().any()]

for col in someButNotAllNulls:
    if ((df[col].value_counts().size < 15) & (df[col].value_counts().size > 1)):
        print(df[col].value_counts().head(2))

for col in someButNotAllNulls:
    if (df[col].value_counts().size == 1):
        print(f"Column can be deleted: {df[col].name}")
        
# Quick observation - columns which has some value and not all nulls
#--------------------------------------------------------------------
# 1. The columns which has one unique value [`collections_12_mths_ex_med`, `chargeoff_within_12_mths`, `tax_liens`]
# 2. The above listed columns can safely be deleted
# 3. The column `next_pymnt_d` doesn't have enough information for analysis and hence can be deleted.
#--------------------------------------------------------------------

Index(['emp_title', 'emp_length', 'desc', 'title', 'mths_since_last_delinq',
       'mths_since_last_record', 'revol_util', 'last_pymnt_d', 'next_pymnt_d',
       'last_credit_pull_d', 'collections_12_mths_ex_med',
       'chargeoff_within_12_mths', 'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')
emp_length
10+ years    8879
< 1 year     4583
Name: count, dtype: int64
next_pymnt_d
Jun-16    1125
Jul-16      15
Name: count, dtype: int64
pub_rec_bankruptcies
0.0    37339
1.0     1674
Name: count, dtype: int64
Column can be deleted: collections_12_mths_ex_med
Column can be deleted: chargeoff_within_12_mths
Column can be deleted: tax_liens


In [ ]:
# Get those column which has no null values
print(df.columns[~df.isnull().any()])

noNulls = df.columns[~df.isnull().any()]

for col in noNulls:
    if ((df[col].value_counts().size < 15) & (df[col].value_counts().size > 1)):
        print(df[col].value_counts().head(2))

for col in noNulls:
    if (df[col].value_counts().size == 1):
        print(f"Column can be deleted: {df[col].name}")
        
# Quick observation - columns with no null values
#-------------------------------------------------
# 1. The columns which has one unique value [pymnt_plan, initial_list_status, policy_code, application_type, acc_now_delinq, delinq_amnt]
# 2. The above listed columns can safely be deleted
#-------------------------------------------------

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'purpose', 'zip_code', 'addr_state',
       'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt', 'policy_code',
       'application_type', 'acc_now_delinq', 'delinq_amnt'],
      dtype='object')
term
36 months    29096
60 months    10621
Name: count, dtype: int64
grade
B    12020
A    10085
Name: count, dtype: int64
home_ownership
RENT        18899
MORTGAGE    17659
Name: count, dtype: int64
verification_status
Not Verified    16921
Verified        12809

## **3. Data Preparation**



### **3.1. Data Cleaning**
- Remove columns
- Handle missing / null values
- Remove duplicates
- Calculate IQR and remove outliers



### **3.2. Data Engineering**
- Create derived columns
- Map categorical variables
- Convert data types



### **3.2. Data Exploration**
- Summarize initial understanding

## **4. Exploratory Data Analysis (EDA)**



### **4.1. Univariate Analysis**



#### **4.1.1. Numeric Variables**
- Histogram for distribution
- Boxplots to find outliers



#### **4.1.2. Categorical Variables**
- Bar charts for frequency distribution



### **4.2. Segmented Univariate Analysis**
- Boxplots by categorical variables



### **4.3. Bivariate Analysis**



#### **4.3.1. Numeric Variables by Numeric Variables**
- Scatter plots
- Correlation heatmaps



#### **4.3.2. Numeric Variables by Categorical Variables**
- Boxplots for comparitive analysis
- Violin plots to find the density



#### **4.3.3. Categorical Variables by Categorical Variables**
- Boxplots grouped by categories
- Heatmaps to find proportions



## **5. Summary and Results**



### **5.1. Summary**
- List the summary of insights leading to loan default



### **5.2. Recommendations**
